In [1]:
# import packages
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pdz
import numpy as np
import pandas as pd

from pandas import Series, DataFrame
import Bio
from Bio import SeqIO,AlignIO

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.mixture import GaussianMixture as GMM

from sklearn.manifold import TSNE
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import copy

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# methods

### 
### This includes code copied and pasted from the main methods used for the website in BioKlustering-Website/BioKlustering/mlmodel/parser/GMM.py
### These methods are copy-pasted instead of directly included due to difficulties importing Django classes for running locally without running the server
###

def parseFasta(data):
    d = {fasta.id : str(fasta.seq) for fasta in SeqIO.parse(data, "fasta")}
    pd.DataFrame([d])
    s = pd.Series(d, name='Sequence')
    s.index.name = 'ID'
    s.reset_index()
    return pd.DataFrame(s)

def get_kmer_table(path,k_min,k_max):
    genes, gene_len, output_df = read_fasta(path)
    count_vect = CountVectorizer(analyzer='char', ngram_range=(k_min, k_max))
    X = count_vect.fit_transform(genes)
    chars = count_vect.get_feature_names_out()
    kmers = X.toarray()
    kmer_freq = []
    for i in range(len(genes)):
        kmer_freq.append(kmers[i] / gene_len[i])
    input = pd.DataFrame(kmer_freq, columns=chars)
    return input, output_df

def get_gene_sequences(filename):
    genes = []
    for record in SeqIO.parse(filename, "fasta"):
        genes.append(str(record.seq))
    return genes

# genes: a list of gene sequences, which can directly be generated from get_gene_sequences().
def get_gene_len(genes):
    gene_len = []

    for i in range(len(genes)):
        gene_len.append(len(genes[i]))
    return gene_len

#read single fasta file containing all the gene sequences
def read_fasta(path):
    all_genes = []
    all_gene_len = []
    output_df = pd.DataFrame()

    virus = parseFasta(path)
    output_df = pd.concat([output_df, virus])
    virus = virus.drop_duplicates(keep="last")
    genes = list(virus['Sequence'])
    genes_seq = get_gene_sequences(path)
    gene_len = get_gene_len(genes_seq)
    all_genes = all_genes + genes_seq
    all_gene_len = all_gene_len + gene_len
    return all_genes, all_gene_len, output_df

def get_predictions(path,k_min,k_max,num_class,cov_type, seed):
    kmer_table, output_df = get_kmer_table(path, k_min, k_max)
    gmm = GMM(n_components=num_class, covariance_type=cov_type, random_state=seed).fit(kmer_table)
    predictions = gmm.predict(kmer_table)
    output_df.insert(0, "Labels", predictions)
    return predictions

def get_predictions_semi(path,k_min,k_max,num_class,cov_type,seed,labels):
    targets = []
    unique_given_labels = get_unique_numbers(labels)
    if num_class < len(unique_given_labels) - 1 and -1 in unique_given_labels:
        num_class = len(unique_given_labels) - 1
    if num_class < len(unique_given_labels) and -1 not in unique_given_labels:
        num_class = len(unique_given_labels)
    kmer_table, output_df = get_kmer_table(path, k_min, k_max)

    finalDf = pd.concat([kmer_table, labels], axis=1)
    gmm = GMM(n_components=num_class, covariance_type=cov_type, random_state=seed)
    for i in range(num_class):
        if i in list(finalDf.Labels):
            targets.append(i)
    if len(targets) == num_class:
        gmm.means_init = np.array([kmer_table[finalDf.Labels == i].mean(axis=0) for i in targets])
    gmm.fit(kmer_table)
    predictions = gmm.predict(kmer_table)

    # Get the counts for the given labels and the predicted labels
    given_labels_count = {}
    labels_list = list(labels)
    for label in unique_given_labels:
        given_labels_count[label] = labels_list.count(label)
    unique_predicted_labels = get_unique_numbers(predictions)
    predicted_labels_count = {}
    for label in unique_predicted_labels:
        predicted_labels_count[label] = (predictions == label).sum()
    max_item = max(predicted_labels_count, key=predicted_labels_count.get)
    if -1 in given_labels_count.keys():
        del given_labels_count[-1]
    given_labels_count = sorted(given_labels_count.items(), key=lambda x: x[1], reverse=True)
    predicted_labels_count = sorted(predicted_labels_count.items(), key=lambda x: x[1], reverse=True)

    res = np.array(predictions)

    # Map the predicted labels to the given/actual labels
    unselected_given = copy.deepcopy(unique_given_labels)
    if -1 in unselected_given:
        unselected_given.remove(-1)
    unselected_pred = copy.deepcopy(unique_predicted_labels)
    map_predict_to_actual = {}
    for label_GIVEN_dict_entry in given_labels_count:
        label_GIVEN = label_GIVEN_dict_entry[0]
        predicted_labels_count_GIVEN = {}
        label_GIVEN_idx = [index for (index, item) in enumerate(labels_list) if item == label_GIVEN ]
        res_GIVEN = [res[i] for i in label_GIVEN_idx]
        unique_predicted_labels_GIVEN = list(set(get_unique_numbers(res_GIVEN)) & set(unselected_pred))
        if len(unique_predicted_labels_GIVEN) == 0:
            continue
        for lab in unique_predicted_labels_GIVEN:
            predicted_labels_count_GIVEN[lab] = (res_GIVEN == lab).sum()
        map_predict_to_actual[max(predicted_labels_count_GIVEN, key=predicted_labels_count_GIVEN.get)] = label_GIVEN
        unselected_given.remove(label_GIVEN)
        to_rem = max(predicted_labels_count_GIVEN, key=predicted_labels_count_GIVEN.get)
        unselected_pred.remove(to_rem)
        print("remove ", label_GIVEN, "from unselected given. ", unselected_given, "remains")
        print("remove ", to_rem, "from unselected pred. ", unselected_pred, "remains")


    max_value = max(unique_given_labels) + 1
    for upl in unique_predicted_labels:
        if upl not in map_predict_to_actual.keys():
            print(f"{upl} mapped to {max_value}")
            map_predict_to_actual[upl] = max_value
            max_value += 1
            unselected_pred.remove(upl)
    
    if len(unselected_given) != len(unselected_pred):
        print("unselected given != unselected pred")
        print("unselected pred: ", unselected_pred)
        print("unselected given: ", unselected_given)
    
    
    print(f"map_predict_to_actual: {map_predict_to_actual}")
    predictions_final = []
    predictions_tmp = []

    # predictions_final contains the final results
    # it takes care of the case when num_class > number of unique labels given
    for i in range(len(predictions)):
        if labels[i] == -1:
            if predictions[i] in map_predict_to_actual.keys():
                predictions_final.append(map_predict_to_actual[predictions[i]])
                predictions_tmp.append(map_predict_to_actual[predictions[i]])
            else:
                predictions_final.append(map_predict_to_actual[max_item])
                predictions_tmp.append(map_predict_to_actual[max_item])
        else:
            predictions_tmp.append(map_predict_to_actual[predictions[i]])
            predictions_final.append(labels[i])

    # get accuracy with regard to known labels

    unknown_label = -1
    total_labeled = 0
    for i in range(len(labels)):
        if labels[i] != unknown_label:
            total_labeled = total_labeled + 1

    correct_count = 0
    temp_accuracy = 0
    for k in range(len(labels)):
        if (labels[k] != unknown_label):
            if (labels[k] == predictions_tmp[k]):
                correct_count += 1
    temp_accuracy = correct_count / total_labeled


    predictions = np.array(predictions_final)
    output_df.insert(0, "Labels", predictions)
    return predictions

def get_unique_numbers(numbers):
    list_of_unique_numbers = []

    unique_numbers = set(numbers)

    for number in unique_numbers:
        list_of_unique_numbers.append(number)

    return list_of_unique_numbers


# Dataset Overview

In [4]:
path = "./concatenated.fasta"
gene,gene_len,ouput_df = read_fasta(path)
sum(gene_len)/len(gene_len)

436428.61475409835

## Unsupervised

In [5]:
from operator import mod

# Define the actual labels
actual_labels = pd.read_csv("./responses-carb.csv")
actual_labels = actual_labels["class"]
actual_labels = actual_labels.tolist()

path = "./concatenated.fasta"
k_min = 3
k_max = 6
num_class = len(np.unique(actual_labels))
cov_type = 'full' 
seed = 1232
predictions1 = get_predictions(path,k_min,k_max,num_class,cov_type,seed).tolist()

In [6]:
import itertools

def get_accuracy(predicted_labels, actual_labels):
    # Generate all possible permutations of label mappings
    possible_mappings = list(itertools.permutations(set(predicted_labels)))

    # Function to calculate accuracy given a label mapping
    def calculate_accuracy(actual_labels, predicted_labels, mapping):
        mapped_labels = [mapping[label] for label in predicted_labels]
        correct_predictions = sum(1 for actual, predicted in zip(actual_labels, mapped_labels) if actual == predicted)
        return correct_predictions / len(actual_labels)

    # Find the mapping with the highest accuracy
    best_mapping = None
    best_accuracy = 0.0

    for mapping in possible_mappings:
        accuracy = calculate_accuracy(actual_labels, predicted_labels, mapping)
        if accuracy > best_accuracy:
            best_mapping = mapping
            best_accuracy = accuracy

    # Create a dictionary containing the best matching and accuracy
    matching_results = {
        'Matching': dict(zip(set(predicted_labels), best_mapping)),
        'Accuracy': best_accuracy
    }

    # Print the results
    return matching_results

best_mapping = get_accuracy(predictions1, actual_labels)
print(best_mapping)

{'Matching': {0: 0}, 'Accuracy': 0.7950819672131147}


In [7]:
predictions1 = [best_mapping['Matching'][label] for label in predictions1]

In [8]:
df_0, output_df = get_kmer_table("./concatenated.fasta",3,3)
df_0.head()

,aaa,aac,aag,aat,aca,acc,acg,act,aga,agc,...,tcg,tct,tga,tgc,tgg,tgt,tta,ttc,ttg,ttt
0,0.003681,0.013174,0.013201,0.003790,0.008649,0.022957,0.016592,0.007628,0.008062,0.022007,...,0.025837,0.006968,0.010662,0.022401,0.024736,0.006024,0.000738,0.013608,0.004279,0.002408
1,0.003801,0.013027,0.013276,0.003833,0.008713,0.022748,0.016381,0.007709,0.008137,0.021706,...,0.026021,0.007188,0.010703,0.022629,0.024683,0.006154,0.000780,0.013976,0.004441,0.002527
2,0.003762,0.012880,0.013292,0.003787,0.008676,0.022743,0.016396,0.007689,0.008113,0.021859,...,0.026054,0.007012,0.010551,0.022643,0.024766,0.006037,0.000734,0.013928,0.004296,0.002454
3,0.003801,0.012784,0.013351,0.003785,0.008708,0.022545,0.016202,0.007648,0.008144,0.021858,...,0.026063,0.007159,0.010539,0.022794,0.024779,0.006092,0.000798,0.013985,0.004483,0.002565
4,0.003836,0.013042,0.013188,0.003879,0.008719,0.022727,0.016411,0.007670,0.008077,0.021893,...,0.025836,0.007073,0.010691,0.022638,0.024780,0.006090,0.000824,0.013791,0.004435,0.002536


In [9]:
number = list(range(1, len(predictions1)+1))
df = pd.DataFrame(list(zip(number, predictions1)), 
               columns =['Number', 'Labels']) 
df.to_csv('gmm_unsup_predictions.csv',index = False)

In [10]:
actual_label = pd.read_csv("./responses-carb.csv")

In [11]:
x = df_0
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents,columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, pd.Series(predictions1)], axis = 1)
finalDf = pd.concat([finalDf, pd.Series(actual_label['class'])], axis = 1)
finalDf.columns = ['principal Component 1', 'Principal Component 2','Predicted Label', 'Actual Label']
finalDf.to_csv('gmm_unsup_pca.csv',index = False)

In [12]:
sum(finalDf['Predicted Label'] == finalDf['Actual Label'])/len(finalDf)

0.7950819672131147

# Semi-supervised

In [13]:
from sklearn.metrics import precision_score

semi_labels = pd.read_csv("./carb-semi-labels.csv")
semi_labels = pd.Series(semi_labels['Labels'])

actual_labels = pd.read_csv("./responses-carb.csv")
actual_labels = actual_labels["class"]
actual_labels = actual_labels.tolist()

path = "./concatenated.fasta"
k_min = 3
k_max = 3
num_class = len(np.unique(actual_labels))
cov_type = 'full' 
seed = 1232
predictions2 = get_predictions_semi(path,k_min,k_max,num_class,cov_type,seed,semi_labels).tolist()

print(F"False positive rate & {precision_score(actual_labels, predictions2, pos_label=1)}")
print(F"False negative rate & {precision_score(actual_labels, predictions2, pos_label=0)}")


remove  0 from unselected given.  [1] remains
remove  0 from unselected pred.  [] remains
unselected given != unselected pred
unselected pred:  []
unselected given:  [1]
map_predict_to_actual: {0: 0}
False positive rate & 1.0
False negative rate & 0.8151260504201681


In [16]:
pd.set_option('display.max_rows', None)
pd.DataFrame(data={'predictions':predictions2,'actual':actual_labels})

,predictions,actual
0,0,1
1,0,0
2,0,1
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [15]:
number = list(range(1, len(predictions2)+1))
df = pd.DataFrame(list(zip(number, predictions2)), 
               columns =['Number', 'Labels']) 
df.to_csv('gmm_semisup_predictions.csv',index = False)